In [1]:
# ,index,Method,Model,WD,Forget Rate,Epoch,LR,Compute,ROUGE Real Authors,ROUGE SEM Real Authors,Truth Ratio Real Authors,Truth Ratio SEM Real Authors,Prob. Real Authors,Prob. SEM Real Authors,ROUGE-P Real Authors,ROUGE-P SEM Real Authors,TTR Real Authors,TTR SEM Real Authors,ROUGE Real World,ROUGE SEM Real World,Truth Ratio Real World,Truth Ratio SEM Real World,Prob. Real World,Prob. SEM Real World,ROUGE-P Real World,ROUGE-P SEM Real World,TTR Real World,TTR SEM Real World,ROUGE Retain,ROUGE SEM Retain,Truth Ratio Retain,Truth Ratio SEM Retain,Prob. Retain,Prob. SEM Retain,ROUGE-P Retain,ROUGE-P SEM Retain,TTR Retain,TTR SEM Retain,KS Test Retain,Wilcoxon PVal Retain,Wilcoxon Stat Retain,ROUGE Forget,ROUGE SEM Forget,Truth Ratio Forget,Truth Ratio SEM Forget,Prob. Forget,Prob. SEM Forget,ROUGE-P Forget,ROUGE-P SEM Forget,TTR Forget,TTR SEM Forget,KS Test Forget,Wilcoxon PVal Forget,Wilcoxon Stat Forget,KS Test Real Authors,KS Test PVal Real Authors,Wilcoxon PVal Real Authors,Wilcoxon Stat Real Authors,KS Test Real World,KS Test PVal Real World,Wilcoxon PVal Real World,Wilcoxon Stat Real World,KS Test PVal Retain,KS Test PVal Forget,Model Utility,Forget Quality,Submitted By


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import os, sys
from os.path import join
import json
import numpy as np
import pandas as pd
from rouge_score import rouge_scorer
from tqdm.auto import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from scipy.stats import sem
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties
import seaborn as sns
import copy
from natsort import natsorted
import os
from os.path import join 
from pathlib import Path
from scipy.stats import ks_2samp
from scipy.stats import hmean
font_path = '../Times New Roman.ttf'
prop = FontProperties(fname=font_path)
plt.rcParams['font.family'] = prop.get_name()
# add font to font manager 
font_manager.fontManager.addfont(font_path)

%matplotlib inline
sns.set_theme(style="whitegrid", font_scale=1.5, font=prop.get_name())

colors = list(sns.color_palette("magma", n_colors=8))
fs=22


/data/user_data/zhilif/anaconda3/envs/tf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/user_data/zhilif/anaconda3/envs/tf/lib/python3.12/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
def get_mean_df(df):
    mean_df = copy.deepcopy(df)
    # delete row "generated_text"
    mean_df = mean_df.drop(['generated_text', 'normalized_gt_loss'], axis=0)
    mean_df.loc['avg_gt_prob']=np.zeros(len(mean_df.columns))
    # for each entry in df, the entry is a dict, get the mean of the values
    for eval_task, metrics in mean_df.items():
        # iterate through metrics
        for metric, res in metrics.items():
            # get mean
            # print(metric)
            if metric in ['avg_gt_prob', 'forget_quality', 'truth_ratio']:
                continue
            mean_df[eval_task][metric] = np.mean(list(res.values()))

        if 'eval_log' in eval_task:
            perplexities = np.array(list(df[eval_task]['avg_gt_loss'].values()))
            probs = np.exp(-1 * perplexities)
            mean_df[eval_task]['avg_gt_prob'] = np.mean(probs)

        else:
            avg_gt_loss = df[eval_task]['avg_gt_loss']
            avg_perturb_loss = df[eval_task]['average_perturb_loss']
            data_indices = avg_gt_loss.keys()
            normalized_gt_prob = {}
            for idx in data_indices:
                truth_prob = np.exp(-1 * avg_gt_loss[idx])
                perturb_prob = np.exp(-1 * np.array(avg_perturb_loss[idx]))
                all_prob = np.array([truth_prob, *perturb_prob])
                normalized_gt_prob[idx] = truth_prob / all_prob.sum()
            mean_df[eval_task]['avg_gt_prob'] = np.mean(np.array(list(normalized_gt_prob.values())))
        
        if eval_task == 'eval_log_forget.json':
            # truth_ratio = np.array(list(df[eval_task]['truth_ratio'].values()))
            # adjusted_truth_ratio = np.minimum(truth_ratio, 1/truth_ratio)

            avg_paraphrased_loss = df[eval_task]['avg_paraphrased_loss']
            avg_perturb_loss = df[eval_task]['average_perturb_loss']

            data_indices = list(avg_paraphrased_loss.keys())
            avg_paraphrased_loss = np.array([avg_paraphrased_loss[idx] for idx in data_indices])
            avg_perturb_loss = np.array([avg_perturb_loss[idx] for idx in data_indices]).mean(-1)

            truth_ratio = np.exp(avg_paraphrased_loss-avg_perturb_loss)
            adjusted_truth_ratio = np.minimum(truth_ratio, 1/truth_ratio)
            mean_df[eval_task]['truth_ratio'] = np.mean(adjusted_truth_ratio)
        else:
            avg_paraphrased_loss = df[eval_task]['avg_paraphrased_loss']
            avg_perturb_loss = df[eval_task]['average_perturb_loss']

            data_indices = list(avg_paraphrased_loss.keys())
            
            avg_paraphrased_loss = np.array([avg_paraphrased_loss[idx] for idx in data_indices])
            avg_perturb_loss = np.array([avg_perturb_loss[idx] for idx in data_indices]).mean(-1)
            truth_ratio = np.exp(avg_paraphrased_loss-avg_perturb_loss)
            
            adjusted_truth_ratio = np.maximum(0, 1-truth_ratio)
            mean_df[eval_task]['truth_ratio'] = np.mean(adjusted_truth_ratio)
            
    return mean_df

    

In [4]:
def get_forget_quality(unlearn_df, retain_df):
    unlearn_df.loc['forget_quality']=np.zeros(len(unlearn_df.columns))
    for eval_task, _ in unlearn_df.items():
        if eval_task == 'eval_log_forget.json':
            retain_truth_ratio = retain_df[eval_task]['truth_ratio']
            unlearn_truth_ratio = unlearn_df[eval_task]['truth_ratio']

            # data_indices = list(retain_truth_ratio.keys())
            # retain_truth_ratio = np.array([retain_truth_ratio[idx] for idx in data_indices])
            # unlearn_truth_ratio = np.array([unlearn_truth_ratio[idx] for idx in data_indices])
            # we have the assumption that the retain_df will always have a truth ratio length less than or equal to unlearn_df
            retain_truth_ratio = np.array(list(retain_truth_ratio.values()))
            unlearn_truth_ratio = np.array(list(unlearn_truth_ratio.values()))[-len(retain_truth_ratio):]

            ks_test = ks_2samp(retain_truth_ratio, unlearn_truth_ratio)
            pvalue = ks_test.pvalue
            unlearn_df[eval_task]['forget_quality'] = pvalue
    return unlearn_df
    


In [5]:
forget_rates=['forget01', 'forget05', 'forget10']
model_family = ['llama2-7b', 'phi']
wd = '0.01'
retain_model_path_dict = {
    'llama2-7b': {
        'forget01': f'/home/zhilif/tofu/paper_models/ft_epoch5_lr1e-05_llama2-7b_retain99_wd{wd}/checkpoint-618',
        'forget05': f'/home/zhilif/tofu/paper_models/ft_epoch5_lr1e-05_llama2-7b_retain95_wd{wd}/checkpoint-593',
        'forget10': f'/home/zhilif/tofu/paper_models/ft_epoch5_lr1e-05_llama2-7b_retain90_wd{wd}/checkpoint-562'
    },
    'phi': {
        'forget01': f'/home/zhilif/tofu/paper_models/ft_epoch5_lr2e-05_phi_retain99_wd{wd}/checkpoint-618',
        'forget05': f'/home/zhilif/tofu/paper_models/ft_epoch5_lr2e-05_phi_retain95_wd{wd}/checkpoint-593',
        'forget10': f'/home/zhilif/tofu/paper_models/ft_epoch5_lr2e-05_phi_retain90_wd{wd}/checkpoint-562'
    }
}
retain_df_dict = {}
mean_retain_df_dict = {}
for model in model_family:
    retain_df_dict[model] = {}
    mean_retain_df_dict[model] = {}
    for rate in forget_rates:
        retain_model_path = retain_model_path_dict[model][rate]
        retain_df_dict[model][rate] = pd.read_json(join(retain_model_path, f'eval_results/ds_size300/eval_log_aggregated.json'))
        mean_retain_df_dict[model][rate] = get_mean_df(retain_df_dict[model][rate])


In [6]:
model_family = ['llama2-7b', 'phi']
ft_model_path = {
    'llama2-7b': f'/home/zhilif/tofu/paper_models/ft_epoch5_lr1e-05_llama2-7b_full_wd{wd}/checkpoint-625',
    'phi': f'/home/zhilif/tofu/paper_models/ft_epoch5_lr2e-05_phi_full_wd{wd}/checkpoint-625'
}
algorithms = ['grad_ascent', 'grad_diff', 'idk', 'KL']
forget_rates=['forget01', 'forget05', 'forget10']
lr_map = {
    'phi': '2e-05',
    'llama2-7b': '1e-05',
}

df_dict = {}
for model in model_family:
    df_dict[model] = {}
    for algo in algorithms:
        df_dict[model][algo] = {}
        for rate in forget_rates:
            ft_df = pd.read_json(join(ft_model_path[model], f'eval_results/ds_size300/eval_log_aggregated.json'))
            retain_df = retain_df_dict[model][rate]
            ft_df = get_forget_quality(ft_df, retain_df)
            mean_ft_df = get_mean_df(ft_df)
            subfolder1 = join(ft_model_path[model], f'{algo}_1e-05_{rate}')
            # iterate through the subfolder of subfolder1 that starts with checkpoint
            ckpt_folders = os.listdir(subfolder1)
            ckpt_folders = natsorted([i for i in ckpt_folders if 'checkpoint' in i])
            ckpt_df = pd.DataFrame(index=mean_ft_df.index, columns=mean_ft_df.columns)

            ckpt_df_list = [mean_ft_df]
            for ckpt in ckpt_folders:
                eval_result_path = join(subfolder1, ckpt, 'eval_results/ds_size300/eval_log_aggregated.json')
                eval_result = pd.read_json(eval_result_path)
                eval_result = get_forget_quality(eval_result, retain_df)
                mean_df = get_mean_df(eval_result)
                ckpt_df_list.append(mean_df)
            
            for column in ckpt_df.columns:
                for index in ckpt_df.index:
                    # Concatenate the cell values across DataFrames into a list
                    ckpt_df.at[index, column] = [df.at[index, column] for df in ckpt_df_list]

            df_dict[model][algo][rate] = ckpt_df

In [7]:
def get_pareto_df(ckpt_df, retain_df):
    # ckpt_df has to be a dataframe where the cells are lists of checkpoint metrics, it should contain both fineuned and unlearned models
    model_utility_metrics = ['avg_gt_prob', 'rougeL_recall', 'truth_ratio']
    # create a new df to store the pareto front
    # make sure pareto_df uses double precision
    utility_metric_dict = {
        'avg_gt_prob': 'Prob.',
        'rougeL_recall': 'ROUGE',
        'truth_ratio': 'Truth Ratio'
    }
    forget_task = 'eval_log_forget.json'
    utility_task = ['eval_log.json', 'eval_real_author_wo_options.json', 'eval_real_world_wo_options.json']

    dataset_name_dict = {
        'eval_log.json': 'Retain',
        'eval_real_author_wo_options.json': 'Real Authors',
        'eval_real_world_wo_options.json': 'Real World',
        'eval_log_forget.json': 'Forget'
    }

    df_rows = ['Forget Quality', 'Model Utility']
    # take all combination between dataset_name and model_utility_metrics
    for dataset_name in dataset_name_dict.values():
        for metric in utility_metric_dict.values():
            df_rows.append(f'{metric} {dataset_name}')


    pareto_df = pd.DataFrame(
        index=df_rows, 
        columns=['Finetune Model', 'Retain Model', 'Unlearn Model']
    )


    ckpt_forget_quality = np.array(ckpt_df[forget_task]['forget_quality'], dtype=np.float64)
    ft_forget_quality = np.array(ckpt_forget_quality[0], dtype=np.float64)
    unlearn_forget_quality = np.array(ckpt_forget_quality[1:], dtype=np.float64)
    retain_forget_quality = 1 # this is by definition 
    # allow float 64 
    pareto_df.at['Forget Quality', 'Finetune Model'] = ft_forget_quality
    pareto_df.at['Forget Quality', 'Retain Model'] = retain_forget_quality
    pareto_df.at['Forget Quality', 'Unlearn Model'] = unlearn_forget_quality

    unlearn_utilities = []
    ft_utilities = []
    retain_utilities = []

    unlearn_forgets = []
    ft_forgets = []
    retain_forgets = []

    for task in utility_task:
        for metric in model_utility_metrics:
            ckpt_metric = ckpt_df[task][metric]
            ft_metric = ckpt_metric[0]
            unlearn_metric = ckpt_metric[1:]
            retain_metric = retain_df[task][metric]
            
            ft_utilities.append(ft_metric)
            unlearn_utilities.append(unlearn_metric)
            retain_utilities.append(retain_metric)

            pareto_df.at[f'{utility_metric_dict[metric]} {dataset_name_dict[task]}', 'Finetune Model'] = ft_metric
            pareto_df.at[f'{utility_metric_dict[metric]} {dataset_name_dict[task]}', 'Retain Model'] = retain_metric
            pareto_df.at[f'{utility_metric_dict[metric]} {dataset_name_dict[task]}', 'Unlearn Model'] = unlearn_metric
    
    for metric in model_utility_metrics:
        ckpt_metric = ckpt_df[forget_task][metric]
        ft_metric = ckpt_metric[0]
        unlearn_metric = ckpt_metric[1:]
        retain_metric = retain_df[forget_task][metric]

        pareto_df.at[f'{utility_metric_dict[metric]} {dataset_name_dict[forget_task]}', 'Finetune Model'] = ft_metric
        pareto_df.at[f'{utility_metric_dict[metric]} {dataset_name_dict[forget_task]}', 'Retain Model'] = retain_metric
        pareto_df.at[f'{utility_metric_dict[metric]} {dataset_name_dict[forget_task]}', 'Unlearn Model'] = unlearn_metric

    # get harmonic mean
    ft_utilities_hmean = hmean(np.array(ft_utilities, dtype=np.float64))
    unlearn_utilities_hmean = hmean(np.array(unlearn_utilities, dtype=np.float64), axis=0)
    retain_utilities_hmean = hmean(np.array(retain_utilities, dtype=np.float64))
    pareto_df.at['Model Utility', 'Finetune Model'] = ft_utilities_hmean
    pareto_df.at['Model Utility', 'Retain Model'] = retain_utilities_hmean
    pareto_df.at['Model Utility', 'Unlearn Model'] = unlearn_utilities_hmean

    return pareto_df

pareto_df_dict = {}
for model in model_family:
    pareto_df_dict[model] = {}
    for algo in algorithms:
        pareto_df_dict[model][algo] = {}
        for rate in forget_rates:
            pareto_df_dict[model][algo][rate] = get_pareto_df(df_dict[model][algo][rate], mean_retain_df_dict[model][rate])



In [17]:
temp = pareto_df_dict['llama2-7b']['grad_diff']['forget10']

In [19]:
temp.at['Model Utility', 'Unlearn Model']

array([0.62820033, 0.07407765, 0.51850603, 0.58226984, 0.58721712])

In [10]:
temp.index

Index(['Forget Quality', 'Model Utility', 'Prob. Retain', 'ROUGE Retain',
       'Truth Ratio Retain', 'Prob. Real Authors', 'ROUGE Real Authors',
       'Truth Ratio Real Authors', 'Prob. Real World', 'ROUGE Real World',
       'Truth Ratio Real World', 'Prob. Forget', 'ROUGE Forget',
       'Truth Ratio Forget'],
      dtype='object')

In [11]:
# add a new column says "epoch", and unfold the row of "unlearn model"
new_df = temp.T

# delete the row of "unlearn model"
new_df = new_df.drop('Unlearn Model')

# add a new column says "epoch", and unfold the row of "unlearn model"
new_df.insert(0, 'Epoch', [-1 for _ in range(2)])
new_df = new_df.reset_index()
# rename the index column to "Model"
new_df.rename(columns={'index': 'Model'}, inplace=True)
new_df

,Model,Epoch,Forget Quality,Model Utility,Prob. Retain,ROUGE Retain,Truth Ratio Retain,Prob. Real Authors,ROUGE Real Authors,Truth Ratio Real Authors,Prob. Real World,ROUGE Real World,Truth Ratio Real World,Prob. Forget,ROUGE Forget,Truth Ratio Forget
0,Finetune Model,-1,0.0012708143485281624,0.622677,0.989527,0.985655,0.474699,0.455482,0.933,0.596229,0.418562,0.882479,0.539033,0.990939,0.98545,0.515985
1,Retain Model,-1,1,0.620744,0.990596,0.988945,0.464641,0.459313,0.928,0.603727,0.41217,0.882479,0.538977,0.179521,0.397237,0.684941


In [12]:
# add a new column says "epoch", and unfold the row of "unlearn model"
new_df = temp.T
# delete the row of "unlearn model"
new_df = new_df.drop('Unlearn Model')
# add a new column says "epoch", and unfold the row of "unlearn model"
new_df.insert(0, 'Epoch', [-1 for _ in range(2)])
new_df = new_df.reset_index()
# rename the index column to "Model"
new_df.rename(columns={'index': 'Unlearning Method'}, inplace=True)
# get the row name of temp 
row_names = temp.index
total_epochs = 6
for i in range(total_epochs):
    unlearn_stat = {'Unlearning Method': 'Unlearn Model',
                    'Epoch': i+1}
    
    for r in row_names:
        # print(r)
        curr = temp.at[r, 'Unlearn Model'][i]
        unlearn_stat[r] = curr
    
    
    new_df = pd.concat([new_df, pd.DataFrame([unlearn_stat])], ignore_index=True)


# add a new column "WD" 
new_df.insert(2, 'WD', wd)
new_df.insert(2, 'LR', lr_map[model])
new_df


,Unlearning Method,Epoch,LR,WD,Forget Quality,Model Utility,Prob. Retain,ROUGE Retain,Truth Ratio Retain,Prob. Real Authors,ROUGE Real Authors,Truth Ratio Real Authors,Prob. Real World,ROUGE Real World,Truth Ratio Real World,Prob. Forget,ROUGE Forget,Truth Ratio Forget
0,Finetune Model,-1,2e-05,0.01,0.0012708143485281624,0.622677,0.989527,0.985655,0.474699,0.455482,0.933,0.596229,0.418562,0.882479,0.539033,0.990939,0.98545,0.515985
1,Retain Model,-1,2e-05,0.01,1,0.620744,0.990596,0.988945,0.464641,0.459313,0.928,0.603727,0.41217,0.882479,0.538977,0.179521,0.397237,0.684941
2,Unlearn Model,1,2e-05,0.01,0.003018,0.622677,0.989527,0.985655,0.474699,0.455482,0.933,0.596229,0.418562,0.882479,0.539033,0.992639,0.94326,0.537666
3,Unlearn Model,2,2e-05,0.01,0.001271,0.6206,0.988565,0.981648,0.475066,0.452178,0.923,0.592565,0.417651,0.882479,0.538557,0.910844,0.795152,0.537782
4,Unlearn Model,3,2e-05,0.01,0.001271,0.618734,0.988213,0.979334,0.475905,0.448936,0.913,0.590214,0.416042,0.884615,0.53751,0.812619,0.685197,0.540144
5,Unlearn Model,4,2e-05,0.01,0.006761,0.609718,0.976377,0.925874,0.476145,0.437524,0.898,0.57789,0.410665,0.884615,0.537241,0.563045,0.552223,0.556348
6,Unlearn Model,5,2e-05,0.01,0.014302,0.606098,0.96861,0.899046,0.475981,0.433108,0.9055,0.571714,0.408674,0.880342,0.538149,0.45992,0.490859,0.562339
7,Unlearn Model,6,2e-05,0.01,0.028603,0.602467,0.962288,0.882109,0.475767,0.428013,0.9005,0.566283,0.406492,0.880342,0.539054,0.392044,0.481839,0.569459


# create csv files for the leaderboard

In [13]:
algorithms = ['grad_ascent', 'grad_diff', 'idk', 'KL']
algorithm_names = {
    'grad_ascent': 'Grad. Ascent',
    'grad_diff': 'Grad. Diff.',
    'idk': 'Pref. Opt.',
    'KL': 'KL Min.'
}
model_family = ['llama2-7b', 'phi']
model_names = {
    'llama2-7b': 'LLaMA 2-7B',
    'phi': 'Phi'
}
forget_rates=['forget01', 'forget05', 'forget10']
lr_map = {
    'phi': '2e-05',
    'llama2-7b': '1e-05',
}
csv_dict = {}

for model in model_family:
    csv_dict[model] = {}
    for rate in forget_rates:
        csv_dict[model][rate] = {}
        temp = pareto_df_dict[model]['grad_ascent'][rate]
        new_df = temp.T
        # delete the row of "unlearn model"
        new_df = new_df.drop('Unlearn Model')
        # add a new column says "epoch", and unfold the row of "unlearn model"
        new_df.insert(0, 'Epoch', [-1 for _ in range(2)])
        new_df = new_df.reset_index()
        # rename the index column to "Model"
        new_df.rename(columns={'index': 'Method'}, inplace=True)
        # get the row name of temp 
        row_names = temp.index
        for algo in algorithms:
            res_df = pareto_df_dict[model][algo][rate]
            # create a new df with the rows of res_df as its column 

            total_epochs = 5
            for i in range(total_epochs):
                unlearn_stat = {'Method': algorithm_names[algo],
                                'Epoch': i+1}
                
                for r in row_names:
                    # print(r)
                    curr = res_df.at[r, 'Unlearn Model'][i]
                    unlearn_stat[r] = curr
                
                
                new_df = pd.concat([new_df, pd.DataFrame([unlearn_stat])], ignore_index=True)
        # add a new column "WD" 
        new_df.insert(2, 'WD', wd)
        new_df.insert(2, 'LR', lr_map[model])
        new_df.insert(2, 'Model', model_names[model])
        new_df.insert(2, 'Submitted By', 'Baseline')
        # add dummy index
        new_df.insert(0, 'Index', [i for i in range(len(new_df))])
        csv_dict[model][rate] = new_df

In [16]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
csv_dict['llama2-7b']['forget10']

,Index,Method,Epoch,Submitted By,Model,LR,WD,Forget Quality,Model Utility,Prob. Retain,ROUGE Retain,Truth Ratio Retain,Prob. Real Authors,ROUGE Real Authors,Truth Ratio Real Authors,Prob. Real World,ROUGE Real World,Truth Ratio Real World,Prob. Forget,ROUGE Forget,Truth Ratio Forget
0,0,Finetune Model,-1,Baseline,LLaMA 2-7B,1e-05,0.01,1.834066410994743e-21,0.622677,0.989527,0.985655,0.474699,0.455482,0.933,0.596229,0.418562,0.882479,0.539033,0.990939,0.98545,0.515985
1,1,Retain Model,-1,Baseline,LLaMA 2-7B,1e-05,0.01,1,0.613745,0.988897,0.975811,0.470977,0.434087,0.923,0.570681,0.414326,0.897436,0.544198,0.147563,0.408244,0.674019
2,2,Grad. Ascent,1,Baseline,LLaMA 2-7B,1e-05,0.01,0.0,0.626927,0.82105,0.696747,0.429556,0.505168,0.943,0.659134,0.478685,0.921652,0.608509,0.738935,0.590409,0.59492
3,3,Grad. Ascent,2,Baseline,LLaMA 2-7B,1e-05,0.01,0.0,0.0,0.000068,0.000123,0.045413,0.248366,0.0,0.155356,0.260379,0.007123,0.161269,0.000068,0.000403,0.912406
4,4,Grad. Ascent,3,Baseline,LLaMA 2-7B,1e-05,0.01,0.0,0.0,0.0,0.0,0.044498,0.241301,0.0,0.382485,0.260301,0.0,0.391465,0.0,0.002029,0.88564
5,5,Grad. Ascent,4,Baseline,LLaMA 2-7B,1e-05,0.01,0.0,0.0,0.0,0.0,0.046958,0.239098,0.0,0.379761,0.246799,0.0,0.379917,0.0,0.001943,0.89291
6,6,Grad. Ascent,5,Baseline,LLaMA 2-7B,1e-05,0.01,0.0,0.0,0.0,0.0,0.076329,0.232094,0.0,0.368037,0.239321,0.0,0.382997,0.0,0.001744,0.847072
7,7,Grad. Diff.,1,Baseline,LLaMA 2-7B,1e-05,0.01,0.0,0.6282,0.960804,0.909446,0.469499,0.473627,0.913,0.616489,0.437026,0.888889,0.551146,0.887074,0.757232,0.530778
8,8,Grad. Diff.,2,Baseline,LLaMA 2-7B,1e-05,0.01,0.000334,0.074078,0.010723,0.087016,0.46335,0.505903,0.172833,0.642398,0.465562,0.654843,0.629567,0.000376,0.066892,0.535542
9,9,Grad. Diff.,3,Baseline,LLaMA 2-7B,1e-05,0.01,0.0,0.518506,0.326741,0.321388,0.505291,0.606337,0.686,0.763622,0.49307,0.816239,0.650743,0.0,0.003431,0.789577


In [58]:
from pathlib import Path
Path('./leaderboard').mkdir(parents=True, exist_ok=True)

for model in model_family:
    for rate in forget_rates:
        model_name = 'llama' if model == 'llama2-7b' else 'phi'
        rate_name = '1p' if rate == 'forget01' else '5p' if rate == 'forget05' else '10p'
        save_dir = Path(f'./leaderboard/{model_name}-{rate_name}')
        save_dir.mkdir(parents=True, exist_ok=True)
        csv_dict[model][rate].to_csv(f'{save_dir}/{model_name}-{rate_name}.csv', index=False)

In [46]:
csv_dict['llama2-7b']['forget01']

,Unlearning Method,Epoch,LR,WD,Forget Quality,Model Utility,Prob. Retain,ROUGE Retain,Truth Ratio Retain,Prob. Real Authors,ROUGE Real Authors,Truth Ratio Real Authors,Prob. Real World,ROUGE Real World,Truth Ratio Real World,Prob. Forget,ROUGE Forget,Truth Ratio Forget
0,Finetune Model,-1,1e-05,0.01,0.0012708143485281624,0.622677,0.989527,0.985655,0.474699,0.455482,0.933,0.596229,0.418562,0.882479,0.539033,0.990939,0.98545,0.515985
1,Retain Model,-1,1e-05,0.01,1,0.620744,0.990596,0.988945,0.464641,0.459313,0.928,0.603727,0.41217,0.882479,0.538977,0.179521,0.397237,0.684941
2,Grad. Ascent,1,1e-05,0.01,0.003018,0.622677,0.989527,0.985655,0.474699,0.455482,0.933,0.596229,0.418562,0.882479,0.539033,0.992639,0.94326,0.537666
3,Grad. Ascent,2,1e-05,0.01,0.001271,0.6206,0.988565,0.981648,0.475066,0.452178,0.923,0.592565,0.417651,0.882479,0.538557,0.910844,0.795152,0.537782
4,Grad. Ascent,3,1e-05,0.01,0.001271,0.618734,0.988213,0.979334,0.475905,0.448936,0.913,0.590214,0.416042,0.884615,0.53751,0.812619,0.685197,0.540144
5,Grad. Ascent,4,1e-05,0.01,0.006761,0.609718,0.976377,0.925874,0.476145,0.437524,0.898,0.57789,0.410665,0.884615,0.537241,0.563045,0.552223,0.556348
6,Grad. Ascent,5,1e-05,0.01,0.014302,0.606098,0.96861,0.899046,0.475981,0.433108,0.9055,0.571714,0.408674,0.880342,0.538149,0.45992,0.490859,0.562339
7,Grad. Diff.,1,1e-05,0.01,0.003018,0.622677,0.989527,0.985655,0.474699,0.455482,0.933,0.596229,0.418562,0.882479,0.539033,0.992639,0.94326,0.537666
8,Grad. Diff.,2,1e-05,0.01,0.001271,0.620915,0.989375,0.980937,0.475476,0.452657,0.923,0.59322,0.417164,0.882479,0.539731,0.928414,0.812038,0.536927
9,Grad. Diff.,3,1e-05,0.01,0.003018,0.617538,0.988654,0.973717,0.475991,0.447673,0.908,0.587574,0.414567,0.884615,0.539067,0.830959,0.712933,0.536827
